In [3]:
import sys
from pathlib import Path
CWD = Path.cwd()
for p in [CWD, CWD.parent, CWD.parent.parent]:
    if (p / "src").exists():
        sys.path.append(str(p / "src"))
        PROJECT_ROOT = p
        break

from utils import get_data_dir, load_local_raw
RAW_DF = load_local_raw(get_data_dir(raw=True))
print("raw shape:", RAW_DF.shape)
RAW_DF.head()

raw shape: (280178, 176)


c:\Users\gwanji\pjt-apt_analysis\src\utils.py:105: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["region_group"] = np.where(df[city_col].isin(KOREA_CAPITAL), "수도권", "지방")


,1,광주광역시 북구 삼각동,835,0835,0000,일곡엘리체프라임,84.9426,202508,14,"41,500",...,제주특별자치도 서귀포시 강정동,195,0195,서귀포강정지구3블럭중흥에스-클래스,84.9297,"48,700",7,대청로 12,제주 서귀포시,region_group
0,2,광주광역시 서구 마륵동,1,1.0,0.0,상무자이,170.8552,202508.0,14,"68,000",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,지방
1,3,광주광역시 서구 화정동,23-20,23.0,20.0,해광샹그릴라센트럴337,54.6212,202508.0,14,"17,900",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,지방
2,4,광주광역시 서구 화정동,859-1,859.0,1.0,현대(859-1),60.0000,202508.0,14,"12,500",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,지방
3,5,광주광역시 광산구 우산동,1625,1625.0,0.0,쌍용더플래티넘광산,84.9784,202508.0,14,"41,300",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,지방
4,6,광주광역시 북구 일곡동,816-3,816.0,3.0,동아2,99.8600,202508.0,14,"30,000",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,지방


In [4]:
import pandas as pd
import numpy as np

def ensure_required_cols(df: pd.DataFrame) -> pd.DataFrame:
    """분석에 필요한 핵심 컬럼들을 안전하게 생성/보정한다."""
    out = df.copy()

    # 1) 원본 한글 컬럼을 표준 이름으로 동기화 (있을 때만)
    col_ko_to_std = {
        "전용면적(㎡)": "area_m2",
        "거래금액(만원)": "price_manwon",
        "계약년월": "yyyymm",
        "계약일": "day",
    }
    for ko, std in col_ko_to_std.items():
        if ko in out.columns and std not in out.columns:
            out[std] = out[ko]

    # 2) 숫자형 보정
    if "price_manwon" in out.columns:
        out["price_manwon"] = (
            out["price_manwon"].astype(str).str.replace(",", "", regex=False).str.strip()
        )
        out["price_manwon"] = pd.to_numeric(out["price_manwon"], errors="coerce")

    if "area_m2" in out.columns:
        out["area_m2"] = pd.to_numeric(out["area_m2"], errors="coerce")

    # 3) 날짜/월 생성
    if "month" not in out.columns:
        if "date" in out.columns and pd.api.types.is_datetime64_any_dtype(out["date"]):
            out["month"] = out["date"].dt.to_period("M").dt.to_timestamp()
        else:
            # yyyymm(+day)로부터 생성 시도
            if "yyyymm" in out.columns:
                yyyymm = out["yyyymm"].astype(str).str.slice(0, 6)
                if "day" in out.columns:
                    day = out["day"].astype(str).str.zfill(2)
                    out["date"] = pd.to_datetime(yyyymm + day, format="%Y%m%d", errors="coerce")
                else:
                    out["date"] = pd.to_datetime(yyyymm + "01", format="%Y%m%d", errors="coerce")
                out["month"] = out["date"].dt.to_period("M").dt.to_timestamp()

    # 4) 단위면적당 가격
    if "price_per_m2_manwon" not in out.columns and {"price_manwon","area_m2"}.issubset(out.columns):
        out["price_per_m2_manwon"] = out["price_manwon"] / out["area_m2"]

    return out

RAW_DF = ensure_required_cols(RAW_DF)

print("필수 컬럼 유무:",
      "month" in RAW_DF.columns,
      "price_per_m2_manwon" in RAW_DF.columns)
RAW_DF.head(2)


필수 컬럼 유무: False False


,1,광주광역시 북구 삼각동,835,0835,0000,일곡엘리체프라임,84.9426,202508,14,"41,500",...,제주특별자치도 서귀포시 강정동,195,0195,서귀포강정지구3블럭중흥에스-클래스,84.9297,"48,700",7,대청로 12,제주 서귀포시,region_group
0,2,광주광역시 서구 마륵동,1,1.0,0.0,상무자이,170.8552,202508.0,14,"68,000",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,지방
1,3,광주광역시 서구 화정동,23-20,23.0,20.0,해광샹그릴라센트럴337,54.6212,202508.0,14,"17,900",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,지방


In [5]:
def rename_columns(df):
    rename_map = {
        '거래금액': 'price_manwon',
        '전용면적': 'area_m2',
        '계약년월': 'contract_ym',
        '계약일': 'contract_day',
        '일자': 'date',  # 일부 파일에 있을 수도 있음
        '지역코드': 'region_code',
    }

    for old_col, new_col in rename_map.items():
        if old_col in df.columns:
            df = df.rename(columns={old_col: new_col})
    
    # 날짜 조합 처리
    if 'contract_ym' in df.columns and 'contract_day' in df.columns:
        df['contract_ym'] = df['contract_ym'].astype(str)
        df['contract_day'] = df['contract_day'].astype(str).str.zfill(2)
        df['date'] = pd.to_datetime(df['contract_ym'] + df['contract_day'], format="%Y%m%d", errors='coerce')

    return df


In [6]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import glob
import pandas as pd

files = glob.glob("data/raw/*.csv")

dfs = []
for f in files:
    try:
        df = pd.read_csv(f, encoding='cp949', sep='\t')
        dfs.append(df)
        print(f"✅ {f} loaded")
    except Exception as e:
        print(f"❌ {f} failed: {e}")


✅ data/raw\광주_아파트_1.csv loaded
✅ data/raw\광주_아파트_2.csv loaded
✅ data/raw\광주_아파트_3.csv loaded
✅ data/raw\광주_아파트_4.csv loaded
✅ data/raw\광주_아파트_5.csv loaded
✅ data/raw\부산_아파트_1.csv loaded
✅ data/raw\부산_아파트_2.csv loaded
✅ data/raw\부산_아파트_3.csv loaded
✅ data/raw\부산_아파트_4.csv loaded
✅ data/raw\부산_아파트_5.csv loaded
✅ data/raw\제주_아파트_1.csv loaded
✅ data/raw\제주_아파트_2.csv loaded
✅ data/raw\제주_아파트_3.csv loaded
✅ data/raw\제주_아파트_4.csv loaded
✅ data/raw\제주_아파트_5.csv loaded


In [8]:
print(dfs[0].columns)

Index(['□ 본 서비스에서 제공하는 정보는 법적인 효력이 없으므로 참고용으로만 활용하시기 바랍니다.'], dtype='object')


In [9]:
import glob
import pandas as pd

files = glob.glob("data/raw/*.csv")

dfs = []
for f in files:
    try:
        df = pd.read_csv(f, encoding='cp949', sep='\t', skiprows=1)
        dfs.append(df)
        print(f"✅ {f} loaded")
    except Exception as e:
        print(f"❌ {f} failed: {e}")


✅ data/raw\광주_아파트_1.csv loaded
✅ data/raw\광주_아파트_2.csv loaded
✅ data/raw\광주_아파트_3.csv loaded
✅ data/raw\광주_아파트_4.csv loaded
✅ data/raw\광주_아파트_5.csv loaded
✅ data/raw\부산_아파트_1.csv loaded
✅ data/raw\부산_아파트_2.csv loaded
✅ data/raw\부산_아파트_3.csv loaded
✅ data/raw\부산_아파트_4.csv loaded
✅ data/raw\부산_아파트_5.csv loaded
✅ data/raw\제주_아파트_1.csv loaded
✅ data/raw\제주_아파트_2.csv loaded
✅ data/raw\제주_아파트_3.csv loaded
✅ data/raw\제주_아파트_4.csv loaded
✅ data/raw\제주_아파트_5.csv loaded


In [10]:
with open("data/raw/광주_아파트_1.csv", encoding='cp949') as f:
    for i in range(20):
        print(f"{i+1:02d}: {f.readline().strip()}")


01: "□ 본 서비스에서 제공하는 정보는 법적인 효력이 없으므로 참고용으로만 활용하시기 바랍니다."
02: "□ 신고정보가 실시간 변경, 해제되어 제공시점에 따라 공개건수 및 내용이 상이할 수 있는 점 참고하시기 바랍니다."
03: "□ 본 자료는 계약일 기준입니다. (※ 7월 계약, 8월 신고건 → 7월 거래건으로  제공)"
04: "□ 통계자료 활용시에는 수치가 왜곡될 수 있으니 참고자료로만 활용하시기  바라며,  외부 공개시에는 반드시 신고일 기준으로 집계되는 공식통계를 이용하여 주시기 바랍니다."
05: ""
06: "* 국토교통부 실거래가 공개시스템의 궁금하신 점이나 문의사항은 콜센터 1533-2949로 연락 주시기 바랍니다."
07: "□ 검색조건"
08: "계약일자 : 2024-08-15 ~ 2025-08-15"
09: "실거래구분 : 아파트(매매)"
10: "주소구분 : 지번주소"
11: "시도 : 광주광역시"
12: "시군구 : 전체"
13: "읍면동 : 전체"
14: "면적 : 전체"
15: "금액선택 : 전체"
16: "NO","시군구","번지","본번","부번","단지명","전용면적(㎡)","계약년월","계약일","거래금액(만원)","동","층","매수자","매도자","건축년도","도로명","해제사유발생일","거래유형","중개사소재지","등기일자"
17: "1","광주광역시 북구 삼각동","835","0835","0000","일곡엘리체프라임","84.9426","202508","14","41,500","-","17","개인","개인","2016","설죽로 419","-","중개거래","광주 북구","-"
18: "2","광주광역시 서구 마륵동","1","0001","0000","상무자이","170.8552","202508","14","68,000","-","4","개인","개인","2008","백석길 22-23","-","중개거래","광주 광산구","-"
19: "3","광주광역시 서구 화정동","23-20","0023","0020","

In [11]:
import glob
import pandas as pd

files = glob.glob("data/raw/*.csv")

dfs = []
for f in files:
    try:
        df = pd.read_csv(f, encoding='cp949', sep='\t', skiprows=15)
        dfs.append(df)
        print(f"✅ {f} loaded")
    except Exception as e:
        print(f"❌ {f} failed: {e}")


✅ data/raw\광주_아파트_1.csv loaded
✅ data/raw\광주_아파트_2.csv loaded
✅ data/raw\광주_아파트_3.csv loaded
✅ data/raw\광주_아파트_4.csv loaded
✅ data/raw\광주_아파트_5.csv loaded
✅ data/raw\부산_아파트_1.csv loaded
✅ data/raw\부산_아파트_2.csv loaded
✅ data/raw\부산_아파트_3.csv loaded
✅ data/raw\부산_아파트_4.csv loaded
✅ data/raw\부산_아파트_5.csv loaded
✅ data/raw\제주_아파트_1.csv loaded
✅ data/raw\제주_아파트_2.csv loaded
✅ data/raw\제주_아파트_3.csv loaded
✅ data/raw\제주_아파트_4.csv loaded
✅ data/raw\제주_아파트_5.csv loaded


In [12]:
print(dfs[0].columns)

Index(['NO,"시군구","번지","본번","부번","단지명","전용면적(㎡)","계약년월","계약일","거래금액(만원)","동","층","매수자","매도자","건축년도","도로명","해제사유발생일","거래유형","중개사소재지","등기일자"'], dtype='object')


In [13]:
column_names = [
    "NO", "시군구", "번지", "본번", "부번", "단지명",
    "전용면적(㎡)", "계약년월", "계약일", "거래금액(만원)",
    "동", "층", "매수자", "매도자", "건축년도", "도로명",
    "해제사유발생일", "거래유형", "중개사소재지", "등기일자"
]

In [14]:
import glob
import pandas as pd

files = glob.glob("data/raw/*.csv")

dfs = []
for f in files:
    try:
        df = pd.read_csv(
            f,
            encoding='cp949',
            # sep='\t',
            skiprows=15,
            # names=column_names
        )
        dfs.append(df)
        print(f"✅ {f} loaded")
    except Exception as e:
        print(f"❌ {f} failed: {e}")

dfs[0]


✅ data/raw\광주_아파트_1.csv loaded
✅ data/raw\광주_아파트_2.csv loaded
✅ data/raw\광주_아파트_3.csv loaded
✅ data/raw\광주_아파트_4.csv loaded
✅ data/raw\광주_아파트_5.csv loaded
✅ data/raw\부산_아파트_1.csv loaded
✅ data/raw\부산_아파트_2.csv loaded
✅ data/raw\부산_아파트_3.csv loaded
✅ data/raw\부산_아파트_4.csv loaded
✅ data/raw\부산_아파트_5.csv loaded
✅ data/raw\제주_아파트_1.csv loaded
✅ data/raw\제주_아파트_2.csv loaded
✅ data/raw\제주_아파트_3.csv loaded
✅ data/raw\제주_아파트_4.csv loaded
✅ data/raw\제주_아파트_5.csv loaded


,NO,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),동,층,매수자,매도자,건축년도,도로명,해제사유발생일,거래유형,중개사소재지,등기일자
0,1,광주광역시 북구 삼각동,835,835,0,일곡엘리체프라임,84.9426,202508,14,"41,500",-,17,개인,개인,2016,설죽로 419,-,중개거래,광주 북구,-
1,2,광주광역시 서구 마륵동,1,1,0,상무자이,170.8552,202508,14,"68,000",-,4,개인,개인,2008,백석길 22-23,-,중개거래,광주 광산구,-
2,3,광주광역시 서구 화정동,23-20,23,20,해광샹그릴라센트럴337,54.6212,202508,14,"17,900",-,18,개인,개인,2017,내방로 337,-,중개거래,광주 서구,-
3,4,광주광역시 서구 화정동,859-1,859,1,현대(859-1),60.0000,202508,14,"12,500",-,6,개인,개인,1987,월드컵4강로 22,-,중개거래,광주 서구,-
4,5,광주광역시 광산구 우산동,1625,1625,0,쌍용더플래티넘광산,84.9784,202508,14,"41,300",-,7,개인,개인,2021,우산로107번길 67,-,중개거래,광주 광산구,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15798,15799,광주광역시 남구 주월동,420-4,420,4,삼다에코빌,81.4048,202408,15,"14,950",삼다에코빌아파트,5,개인,개인,2007,광복마을6길 25,-,중개거래,광주 북구,24.09.25
15799,15800,광주광역시 서구 치평동,1331,1331,0,상무에스케이뷰,59.9671,202408,15,"49,700",101동,16,개인,개인,2016,마륵복개로 140,-,중개거래,광주 서구,24.10.15
15800,15801,광주광역시 서구 치평동,1168-1,1168,1,중흥(1168-1),84.9300,202408,15,"42,500",108,10,개인,개인,1997,치평로 77,-,중개거래,광주 서구,24.10.31
15801,15802,광주광역시 서구 풍암동,1101,1101,0,모아엘가,109.4500,202408,15,"42,000",101,6,개인,개인,2000,풍암중앙로 25,-,중개거래,광주 서구,24.11.15


In [15]:
print(dfs[0].head())


   NO            시군구     번지    본번  부번           단지명   전용면적(㎡)    계약년월  계약일  \
0   1   광주광역시 북구 삼각동    835   835   0      일곡엘리체프라임   84.9426  202508   14   
1   2   광주광역시 서구 마륵동      1     1   0          상무자이  170.8552  202508   14   
2   3   광주광역시 서구 화정동  23-20    23  20  해광샹그릴라센트럴337   54.6212  202508   14   
3   4   광주광역시 서구 화정동  859-1   859   1     현대(859-1)   60.0000  202508   14   
4   5  광주광역시 광산구 우산동   1625  1625   0     쌍용더플래티넘광산   84.9784  202508   14   

  거래금액(만원)  동   층 매수자 매도자  건축년도          도로명 해제사유발생일  거래유형  중개사소재지 등기일자  
0   41,500  -  17  개인  개인  2016      설죽로 419       -  중개거래   광주 북구    -  
1   68,000  -   4  개인  개인  2008    백석길 22-23       -  중개거래  광주 광산구    -  
2   17,900  -  18  개인  개인  2017      내방로 337       -  중개거래   광주 서구    -  
3   12,500  -   6  개인  개인  1987    월드컵4강로 22       -  중개거래   광주 서구    -  
4   41,300  -   7  개인  개인  2021  우산로107번길 67       -  중개거래  광주 광산구    -  


In [16]:
import glob
import pandas as pd

files = glob.glob("data/raw/*.csv")

dfs = []
for f in files:
    try:
        df = pd.read_csv(
            f,
            encoding='cp949',
            sep='\t',
            skiprows=15,
            header=0,
            quotechar='"'
        )
        dfs.append(df)
        print(f"✅ {f} loaded")
    except Exception as e:
        print(f"❌ {f} failed: {e}")


✅ data/raw\광주_아파트_1.csv loaded
✅ data/raw\광주_아파트_2.csv loaded
✅ data/raw\광주_아파트_3.csv loaded
✅ data/raw\광주_아파트_4.csv loaded
✅ data/raw\광주_아파트_5.csv loaded
✅ data/raw\부산_아파트_1.csv loaded
✅ data/raw\부산_아파트_2.csv loaded
✅ data/raw\부산_아파트_3.csv loaded
✅ data/raw\부산_아파트_4.csv loaded
✅ data/raw\부산_아파트_5.csv loaded
✅ data/raw\제주_아파트_1.csv loaded
✅ data/raw\제주_아파트_2.csv loaded
✅ data/raw\제주_아파트_3.csv loaded
✅ data/raw\제주_아파트_4.csv loaded
✅ data/raw\제주_아파트_5.csv loaded


In [17]:
print(dfs[0].columns)


Index(['NO,"시군구","번지","본번","부번","단지명","전용면적(㎡)","계약년월","계약일","거래금액(만원)","동","층","매수자","매도자","건축년도","도로명","해제사유발생일","거래유형","중개사소재지","등기일자"'], dtype='object')


In [18]:
def rename_and_clean(df):
    # ✅ 컬럼명 정리 먼저
    df.columns = df.columns.str.strip().str.replace('"', '').str.replace("'", "")
    # ✅ 컬럼명 매핑
    rename_map = {
        '시군구': 'city',
        '전용면적(㎡)': 'area_m2',
        '거래금액(만원)': 'price_manwon',
        '계약년월': 'contract_ym',
        '계약일': 'contract_day'
    }

    df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})

    # ✅ 필수 컬럼 체크
    required = ['price_manwon', 'area_m2', 'contract_ym', 'contract_day']
    for col in required:
        if col not in df.columns:
            raise KeyError(col)

    # ✅ 숫자 변환
    df['price_manwon'] = df['price_manwon'].astype(str).str.replace(",", "").str.strip()
    df['price_manwon'] = pd.to_numeric(df['price_manwon'], errors="coerce")
    df['area_m2'] = pd.to_numeric(df['area_m2'], errors="coerce")

    # ✅ 날짜 처리
    df['contract_ym'] = df['contract_ym'].astype(str)
    df['contract_day'] = df['contract_day'].astype(str).str.zfill(2)
    df['date'] = pd.to_datetime(df['contract_ym'] + df['contract_day'], format="%Y%m%d", errors='coerce')
    df['month'] = df['date'].dt.to_period("M").dt.to_timestamp()

    # ✅ 평당가 계산
    df['price_per_m2_manwon'] = df['price_manwon'] / df['area_m2']

    return df


In [19]:
clean_dfs = []
for i, df in enumerate(dfs):
    try:
        print(f"▶ Cleaning df[{i}] ...")
        cleaned = rename_and_clean(df)        
        clean_dfs.append(cleaned)
        print(f"✅ df[{i}] cleaned")
    except KeyError as e:
        print(f"❌ df[{i}] skipped - missing column: {e}")
    except Exception as e:
        print(f"❌ df[{i}] failed: {e}")


▶ Cleaning df[0] ...
❌ df[0] skipped - missing column: 'price_manwon'
▶ Cleaning df[1] ...
❌ df[1] skipped - missing column: 'price_manwon'
▶ Cleaning df[2] ...
❌ df[2] skipped - missing column: 'price_manwon'
▶ Cleaning df[3] ...
❌ df[3] skipped - missing column: 'price_manwon'
▶ Cleaning df[4] ...
❌ df[4] skipped - missing column: 'price_manwon'
▶ Cleaning df[5] ...
❌ df[5] skipped - missing column: 'price_manwon'
▶ Cleaning df[6] ...
❌ df[6] skipped - missing column: 'price_manwon'
▶ Cleaning df[7] ...
❌ df[7] skipped - missing column: 'price_manwon'
▶ Cleaning df[8] ...
❌ df[8] skipped - missing column: 'price_manwon'
▶ Cleaning df[9] ...
❌ df[9] skipped - missing column: 'price_manwon'
▶ Cleaning df[10] ...
❌ df[10] skipped - missing column: 'price_manwon'
▶ Cleaning df[11] ...
❌ df[11] skipped - missing column: 'price_manwon'
▶ Cleaning df[12] ...
❌ df[12] skipped - missing column: 'price_manwon'
▶ Cleaning df[13] ...
❌ df[13] skipped - missing column: 'price_manwon'
▶ Cleaning d

In [22]:
import pandas as pd
import numpy as np

# ---------------------------------------------
# 1) 유틸: 안전 표준화(z-score), 선형추세 R^2
# ---------------------------------------------
def _z(x):
    # 표준편차가 0인 경우(단일값) 안전 처리
    s = x.std(ddof=0)
    if pd.isna(s) or s == 0:
        return pd.Series(0.0, index=x.index)
    return (x - x.mean()) / s

def _trend_r2(y: pd.Series):
    """
    단순선형회귀로 시계열 'y'의 추세 적합도 R^2 계산.
    y는 시간순 정렬된 시리즈(길이>=3 권장).
    """
    y = y.dropna()
    n = len(y)
    if n < 3:
        return np.nan
    x = np.arange(n, dtype=float)
    # 최소제곱 직선 적합
    # y_hat = a*x + b
    a, b = np.polyfit(x, y.values, 1)
    y_hat = a * x + b
    ss_res = np.sum((y.values - y_hat) ** 2)
    ss_tot = np.sum((y.values - y.values.mean()) ** 2)
    if ss_tot == 0:
        return np.nan
    return 1 - ss_res / ss_tot

# ---------------------------------------------
# 2) 월별 지역 요약 만들기
# ---------------------------------------------
def build_city_monthly(dfs: list[pd.DataFrame]) -> pd.DataFrame:
    """
    clean_dfs 리스트를 받아 월별(도시×월) 요약을 만든다.
    요구 컬럼: city, price_manwon, area_m2, month, price_per_m2_manwon
    """
    df = pd.concat(dfs, ignore_index=True)
    need_cols = {"city", "price_manwon", "area_m2", "month", "price_per_m2_manwon"}
    missing = need_cols - set(df.columns)
    if missing:
        raise KeyError(f"필수 컬럼 누락: {missing}")

    # 유효값 필터
    df = df.copy()
    df = df[df["city"].notna() & df["month"].notna()]
    df["month"] = pd.to_datetime(df["month"])

    # 월별 지역 집계
    monthly = (
        df.groupby(["city", "month"])
          .agg(
              median_price_manwon=("price_manwon", "median"),
              median_ppm2=("price_per_m2_manwon", "median"),
              tx_count=("price_manwon", "size"),
              median_area_m2=("area_m2", "median"),
          )
          .sort_index()
          .reset_index()
    )

    # 월간 수익률(㎡당 중위가 기준)
    monthly["ret"] = (
        monthly.sort_values(["city", "month"])
               .groupby("city")["median_ppm2"]
               .pct_change()
    )

    return monthly

# ---------------------------------------------
# 3) 최근 윈도우(기본 12개월)로 특성 계산
# ---------------------------------------------
def compute_city_features(monthly: pd.DataFrame,
                          window_months: int = 12,
                          min_months: int = 6) -> pd.DataFrame:
    """
    city×월 데이터에서 최근 window_months 기간을 기준으로
    모멘텀/변동성/유동성/접근성/추세R^2 등 특성 계산.
    """
    monthly = monthly.copy()
    monthly["month"] = pd.to_datetime(monthly["month"])

    feats = []
    for city, g in monthly.groupby("city"):
        g = g.sort_values("month")
        if len(g) < min_months:
            continue

        last_month = g["month"].max()
        start_cut = last_month - pd.DateOffset(months=window_months-1)
        recent = g[g["month"] >= start_cut]

        if len(recent) < min_months:
            continue

        # 최신 값들
        latest_ppm2 = recent.loc[recent["month"].idxmax(), "median_ppm2"]

        # 모멘텀(3M, YoY)
        # 3개월 전
        base_3m = recent[recent["month"] == (last_month - pd.DateOffset(months=3))]
        mom_3m = np.nan
        if not base_3m.empty and latest_ppm2 > 0:
            mom_3m = (latest_ppm2 / base_3m["median_ppm2"].iloc[0]) - 1

        # 12개월 전
        base_12m = g[g["month"] == (last_month - pd.DateOffset(months=12))]
        yoy = np.nan
        if not base_12m.empty and latest_ppm2 > 0:
            yoy = (latest_ppm2 / base_12m["median_ppm2"].iloc[0]) - 1

        # 변동성(최근 기간 ret 표준편차)
        vol = recent["ret"].dropna().std(ddof=0)

        # 유동성(월평균 거래수)
        liq = recent["tx_count"].mean()

        # 접근성(최신 ㎡당 중위가: 낮을수록 좋음)
        affordability = latest_ppm2

        # 추세 적합도 R^2
        r2 = _trend_r2(recent.set_index("month")["median_ppm2"])

        feats.append({
            "city": city,
            "last_month": last_month,
            "mom_3m": mom_3m,
            "yoy": yoy,
            "vol": vol,
            "liquidity": liq,
            "affordability": affordability,
            "trend_r2": r2,
            "obs_months": len(recent),
            "median_area_m2_latest": recent.loc[recent["month"].idxmax(), "median_area_m2"],
            "median_price_manwon_latest": recent.loc[recent["month"].idxmax(), "median_price_manwon"],
        })

    return pd.DataFrame(feats)

# ---------------------------------------------
# 4) 투자 매력도 점수 산출
# ---------------------------------------------
def score_investment_attractiveness(feat_df: pd.DataFrame,
                                    weights: dict | None = None) -> pd.DataFrame:
    """
    특성표를 받아 z-score 표준화 후 가중합 점수 계산.
    낮을수록 좋은 지표(vol, affordability)는 자동 음수 처리.
    """
    if feat_df.empty:
        return feat_df.copy()

    use_weights = {
        "mom_3m": 0.35,
        "yoy": 0.25,
        "vol": 0.15,          # 낮을수록 좋음 → 음수 부호 적용
        "liquidity": 0.15,
        "affordability": 0.05,# 낮을수록 좋음 → 음수 부호 적용
        "trend_r2": 0.05,
    }
    if weights:
        use_weights.update(weights)

    out = feat_df.copy()

    # 각 지표 z-score
    for col in ["mom_3m", "yoy", "vol", "liquidity", "affordability", "trend_r2"]:
        out[f"z_{col}"] = _z(out[col].astype(float))

    # 낮을수록 좋은 지표는 부호 반전
    out["z_vol"] = -out["z_vol"]
    out["z_affordability"] = -out["z_affordability"]

    # 가중합 점수
    out["score"] = (
        use_weights["mom_3m"] * out["z_mom_3m"] +
        use_weights["yoy"] * out["z_yoy"] +
        use_weights["vol"] * out["z_vol"] +
        use_weights["liquidity"] * out["z_liquidity"] +
        use_weights["affordability"] * out["z_affordability"] +
        use_weights["trend_r2"] * out["z_trend_r2"]
    )

    # 순위 및 등급
    out = out.sort_values("score", ascending=False).reset_index(drop=True)
    out["rank"] = np.arange(1, len(out) + 1)
    # 상위 20% A, 20~50% B, 50~80% C, 하위 20% D
    q = out["score"].quantile([0.8, 0.5, 0.2]).to_dict()
    def to_tier(s):
        if s >= q[0.8]: return "A"
        if s >= q[0.5]: return "B"
        if s >= q[0.2]: return "C"
        return "D"
    out["tier"] = out["score"].apply(to_tier)

    # 보기 좋은 컬럼 순서
    cols = [
        "rank", "city", "score", "tier", "last_month", "obs_months",
        "mom_3m", "yoy", "vol", "liquidity", "affordability", "trend_r2",
        "median_price_manwon_latest", "median_area_m2_latest"
    ]
    return out[cols]

# ---------------------------------------------
# 5) 통합 실행 함수
# ---------------------------------------------
def evaluate_investment_attractiveness(clean_dfs: list[pd.DataFrame],
                                       window_months: int = 12,
                                       min_months: int = 6,
                                       weights: dict | None = None):
    """
    입력: 표준화된 clean_dfs 리스트
    출력:
      - scores: 지역별 투자 매력도 결과표(정렬/등급 포함)
      - monthly: 도시×월 요약(검증/시각화용)
      - features: 원시 특성표(점수 계산 전)
    """
    monthly = build_city_monthly(clean_dfs)
    features = compute_city_features(monthly, window_months=window_months, min_months=min_months)
    scores = score_investment_attractiveness(features, weights=weights)
    return scores, monthly, features

# ---------------------------------------------
# 6) 사용 예시
# ---------------------------------------------
# scores, monthly, features = evaluate_investment_attractiveness(
#     clean_dfs,
#     window_months=12,
#     min_months=6,
#     weights={
#         "mom_3m": 0.35,
#         "yoy": 0.25,
#         "vol": 0.15,
#         "liquidity": 0.15,
#         "affordability": 0.05,
#         "trend_r2": 0.05,
#     }
# )
# print(scores.head(15))
# scores.to_csv("output/city_investment_scores.csv", index=False, encoding="utf-8-sig")
